In [21]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time

from dotenv import load_dotenv
import os

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from azure.ai.vision.imageanalysis.models import VisualFeatures

from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time
import matplotlib.pyplot as plt
from skimage import io
import numpy as np

# Încarcă variabilele de mediu din fișierul .env
load_dotenv()

'''
Authenticate
Authenticates your credentials and creates a client.
'''
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
'''
END - Authenticate
'''


def returneazaCoordonateleCadranuluiBicicletei(imagePath):
    """ 
    returneaza coordonatele cadranului unei biciclete
    """
     # automatically identify the object location
    img = open(imagePath, "rb")
    predicted_bicyle_bb = [0,0,0,0]
    result = computervision_client.analyze_image_in_stream(img, visual_features=[VisualFeatureTypes.objects])
    for ob in result.objects:
        if (ob.object_property == "bicycle"):
            predicted_bicyle_bb = [ob.rectangle.x, ob.rectangle.y, ob.rectangle.x + ob.rectangle.w, ob.rectangle.y + ob.rectangle.h]
    return predicted_bicyle_bb


def afisareCadranePrezise():
    """ 
    Pb 2a,b
    Afiseaza imaginile si bicicletele detectate
    """
    fileReader = open("ground_truth.csv")
    lines = fileReader.readlines()

    for line in lines:
        line.removesuffix('\n')
        tokens = line.split(",")
        imagePath = tokens[0]
        x_top_left = int(tokens[1])
        y_top_left = int(tokens[2])

        x_bottom_right = int(tokens[3])
        y_bottom_right = int(tokens[4])

        image = Image.open(imagePath)
        coordonates = returneazaCoordonateleCadranuluiBicicletei(imagePath)

        fig, ax = plt.subplots()
        ax.imshow(image)
        ax.add_patch(plt.Rectangle((coordonates[0], coordonates[1]), coordonates[2] - coordonates[0], coordonates[3] - coordonates[1], fill = False, color = "blue", linewidth = 2))
        ax.add_patch(plt.Rectangle((x_top_left, y_top_left), x_bottom_right - x_top_left, y_bottom_right - y_top_left, fill = False, color = "green", linewidth = 2))
    plt.show()


def clasificationPerformance():
    """
    Pt pb 1
    Returneaza TN (True Negative), FP(False Positive), FN(False Negative), TP(True Positive)
    """
    fileReader = open("ground_truth.csv")
    lines = fileReader.readlines()

    TN = 0
    FP = 0
    FN = 0
    TP = 0
    
    for line in lines:
        line.removesuffix('\n')
        tokens = line.split(",")
        imagePath = tokens[0]
        x_top_left_ground_truth = int(tokens[1])
        y_top_left_ground_truth = int(tokens[2])

        x_bottom_right_ground_truth = int(tokens[3])
        y_bottom_right_ground_truth = int(tokens[4])

        image = Image.open(imagePath)
        coordonates = returneazaCoordonateleCadranuluiBicicletei(imagePath)

        #daca coordonatele ground_truth sunt toate 0 am decis sa consider ca imaginea respectiva nu contine bicicleta
        if x_top_left_ground_truth == y_top_left_ground_truth == x_bottom_right_ground_truth == y_bottom_right_ground_truth == 0:
            if coordonates[0] == coordonates[1] == coordonates[2] == coordonates[3] == 0:
                TN += 1
            else:
                FP += 1
        else:
            if coordonates[0] == coordonates[1] == coordonates[2] == coordonates[3] == 0:
                FN += 1
            else:
                TP += 1
    
    return TN, FP, FN, TP


def getAccuracy(TN, FP, FN, TP):
    """ 
    accuracy represents the overall performance of classification model:
    (TP+TN)/(TN+FP+FN+TP)
    """
    return (TP+TN)/(TN+FP+FN+TP)

def getPrecision(FP, TP):
    """
    precision indicates how accurate the positive predictions are 
    TP/(TP+FP)
    """
    return TP/(TP+FP)

def getRecall(TP, FN):
    """ 
    recall indicates the coverage of actual positive sample
    TP/(TP+FN)
    """
    return TP/(TP+FN)


def rectangleArea(top_left, bottom_right):
    """ 
    Returns the area of a rectangle with the given top left and bottom right point coordonates
    """
    height = bottom_right[1] - top_left[1]
    width = bottom_right[0]-top_left[0]
    area = height * width
    return area


def intersectionArea(top_left, bottom_right, expected_top_left, expected_bottom_right):
    """ 
    returns the intersection area of the two areas defined by the given point coordonates
    """
    intersection_top_left = (max(top_left[0], expected_top_left[0]), max(top_left[1], expected_top_left[1]))
    intersection_bottom_right = (min(bottom_right[0], expected_bottom_right[0]), min(bottom_right[1], expected_bottom_right[1]))

    intersection_area = rectangleArea(intersection_top_left, intersection_bottom_right)
    return intersection_area

def unionArea(top_left, bottom_right, expected_top_left, expected_bottom_right):
    """ 
    returns the union area of the two areas defined by the given point coordonates
    """
    computedArea = rectangleArea(top_left, bottom_right)
    expectedArea = rectangleArea(expected_top_left, expected_bottom_right)
    intersection_area = intersectionArea(top_left, bottom_right, expected_top_left, expected_bottom_right)
    union_area = computedArea + expectedArea - intersection_area
    return union_area

def intersectionOverUnion(top_left, bottom_right, expected_top_left, expected_bottom_right):
    """ 
    pb 2c
    determine performanta algoritmului de la punctul a avand in vedere etichetarile realizate
    """
    if top_left == bottom_right == expected_top_left == expected_bottom_right == (0,0):
        intersection_over_union = 1.0
    else:
        intersection_area = intersectionArea(top_left, bottom_right, expected_top_left, expected_bottom_right)
        union_area = unionArea(top_left, bottom_right, expected_top_left, expected_bottom_right)
        intersection_over_union = intersection_area / union_area
    return intersection_over_union

def printIntersectionOverUnionForAllImages():
    fileReader = open("ground_truth.csv")
    lines = fileReader.readlines()

    for line in lines:
        line.removesuffix('\n')
        tokens = line.split(",")
        imagePath = tokens[0]
        x_top_left_ground_truth = int(tokens[1])
        y_top_left_ground_truth = int(tokens[2])

        x_bottom_right_ground_truth = int(tokens[3])
        y_bottom_right_ground_truth = int(tokens[4])

        top_left_ground_truth = (x_top_left_ground_truth, y_top_left_ground_truth)
        bottom_right_ground_truth = (x_bottom_right_ground_truth, y_bottom_right_ground_truth)

        image = Image.open(imagePath)
        coordonates = returneazaCoordonateleCadranuluiBicicletei(imagePath)
        top_left = (coordonates[0], coordonates[1])
        bottom_right = (coordonates[2], coordonates[3])

        intersection_over_union = intersectionOverUnion(top_left, bottom_right, top_left_ground_truth, bottom_right_ground_truth)
        print("Image ", imagePath, " intersection over union = ", intersection_over_union)
    

def algorithmPerformancePb2c(threshold):
    """
    Pt pb 2c
    Returneaza TN (True Negative), FP(False Positive), FN(False Negative), TP(True Positive)
    """
    fileReader = open("ground_truth.csv")
    lines = fileReader.readlines()

    TN = 0
    FP = 0
    FN = 0
    TP = 0
    
    for line in lines:
        line.removesuffix('\n')
        tokens = line.split(",")
        imagePath = tokens[0]
        x_top_left_ground_truth = int(tokens[1])
        y_top_left_ground_truth = int(tokens[2])

        x_bottom_right_ground_truth = int(tokens[3])
        y_bottom_right_ground_truth = int(tokens[4])

        top_left_ground_truth = (x_top_left_ground_truth, y_top_left_ground_truth)
        bottom_right_ground_truth = (x_bottom_right_ground_truth, y_bottom_right_ground_truth)

        image = Image.open(imagePath)
        coordonates = returneazaCoordonateleCadranuluiBicicletei(imagePath)
        top_left = (coordonates[0], coordonates[1])
        bottom_right = (coordonates[2], coordonates[3])

        intersection_over_union = intersectionOverUnion(top_left, bottom_right, top_left_ground_truth, bottom_right_ground_truth)

        #daca coordonatele ground_truth sunt toate 0 am decis sa consider ca imaginea respectiva nu contine bicicleta
        if x_top_left_ground_truth == y_top_left_ground_truth == x_bottom_right_ground_truth == y_bottom_right_ground_truth == 0:
            if intersection_over_union < threshold:
                TN += 1
            else:
                FP += 1
        else:
            if intersection_over_union < threshold:
                FN += 1
            else:
                TP += 1
    
    return TN, FP, FN, TP



In [27]:
def main():
    #Care task dureaza mai mult (cel de la punctul a sau cel de la punctul b)?
    #Dureaza mai mult sa determinam noi chenarul
    
    # afisareCadranePrezise()

    # TN, FP, FN, TP = clasificationPerformance()
    # accuracy = getAccuracy(TN, FP, FN, TP)
    # precision = getPrecision(FP, TP)
    # recall = getRecall(TP, FN)

    # print("Acuratetea = ", accuracy)
    # print("Precizia = ", precision)
    # print("Rapel = ", recall)

    # printIntersectionOverUnionForAllImages()

    threshold = 0.5
    TN, FP, FN, TP = algorithmPerformancePb2c(threshold)
    accuracy = getAccuracy(TN, FP, FN, TP)
    precision = getPrecision(FP, TP)
    recall = getRecall(TP, FN)

    print("Acuratetea = ", accuracy)
    print("Precizia = ", precision)
    print("Rapel = ", recall)
    

main()

Acuratetea =  0.4
Precizia =  0.4444444444444444
Rapel =  0.8
